Montar o Google Drive no sistema de arquivos

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

# mudando o diretório atual
%cd /gdrive/My Drive/ERCAS_2019

**Extração de características:**



In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
#from keras.backend import clear_session

def vgg16_f_extractor(input_data, layer_name='fc2'):    

    # CNN VGG-16 pré-treinada com os pesos da imagenet
    vgg16_model = VGG16(weights='imagenet', include_top=True)

    # modelo 'cortado' antes da camada de classificação
    ext_model = Model(inputs=vgg16_model.input,
                      outputs=vgg16_model.get_layer(layer_name).output)  # exclusão da última camada da VGG-16

    input_data = preprocess_input(input_data)

    # o método predict() de ext_model NÃO dá a classificação como saída, e sim o vetor de atributos de saída da camada layer_name
    features = ext_model.predict(input_data,verbose=1)
    #clear_session()
    return features

In [0]:
# pequeno teste da extração
import numpy as np
data=np.zeros((10,224,224,3))
features=vgg16_f_extractor(data)

print(data.shape)
print(features.shape)

Teste de feature extraction com base de imagens em disco

In [0]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.datasets import load_files
from skimage.io import imread
from skimage.transform import resize

train_dir='./catdog_small/training_set/'
test_dir ='./catdog_small/test_set/'

# função que lê as imagens de um diretório com classes nos subdiretórios
def load_images(folder,shuffle=True,shape=(224,224)):
    data=load_files(folder,load_content=False, shuffle=shuffle)
    fnames,labels = data['filenames'],data['target']
    x=[]
    for f in fnames:
        print(len(x),'\r')
        img=imread(f)
        x.append(resize(img,output_shape=shape))
    return np.array(x),labels

x_train_imgs,y_train=load_images(train_dir)
x_test_imgs,y_test=load_images(test_dir)

#extração de características com base nas imagens
x_train=vgg16_f_extractor(x_train_imgs)
x_test=vgg16_f_extractor(x_test_imgs)

print(x_test.shape,y_test.shape)

knn_classifier=KNN(n_neighbors=15)
knn_classifier.fit(x_train,y_train)
knn_classifier.score(x_test,y_test)

**Ajuste Fino**

In [0]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import SGD

# CNN VGG-16 pré-treinada com os pesos da imagenet
vgg16_model = VGG16(weights='imagenet', include_top=True)

# guardar a penúltima camada (excluida a de classificação)
fc2=vgg16_model.get_layer('fc2')

#nova camada de classificação binária
newlayer=Dense(2, activation='softmax')(fc2.output)

#modelo com a nova camada de classificação
ft_model = Model(inputs=vgg16_model.input, outputs=newlayer)

#fixar (freeze) pesos de algumas camadas
for l in ft_model.layers[:-2]:
    l.trainable=False

#compilação do modelo
ft_model.compile(optimizer=SGD(lr=0.001,
                               decay=1e-6,
                               momentum=0.9,
                               nesterov=True),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

ft_model.summary()

Treino e teste com base de dados em pasta local

**ImageDataGenerator.flow_from_directory** percorre a pasta de imagens sem alocá-la completamente na memória

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator()

train_iterator=datagen.flow_from_directory(directory='./catdog_small/training_set/',
                                           target_size=(224,224),
                                           batch_size=32,
                                           shuffle=True)
test_iterator=datagen.flow_from_directory(directory='./catdog_small/test_set/',
                                          target_size=(224,224),
                                           batch_size=32,
                                           shuffle=False)
'''
# demonstração -> verificação dos shapes
for x,y in test_iterator:
    print(x.shape, y.shape, y[0])
'''

train_metrics = ft_model.evaluate_generator(train_iterator,verbose=1)
test_metrics = ft_model.evaluate_generator(test_iterator,verbose=1)
print('Antes(train_set):',train_metrics)
print('Antes(test_set): ',test_metrics)

# treinamento (ajuste fino das camadas treináveis)
ft_model.fit_generator(generator=train_iterator,
                       validation_data=test_iterator,
                       epochs=80,verbose=1)